In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.distributions as dist
import biogeme.models as models
import biogeme.results as res

In [2]:
# data = pd.read_csv('./../all_answers.dat',sep='\t', decimal='.')
# data = pd.read_csv('./../log_data.dat',sep='\t', decimal='.')
data = pd.read_csv('./../log_data_sin-filtrar.dat',sep='\t', decimal='.')

In [3]:
data = data[(data['question'] == 2) & (data.gender != 99)]
data.loc[:,'choice'] = data.loc[:,'choice'].replace({99:3})

In [4]:
data.describe().T
# .iloc[27:,:]
# for i in data.columns:
#     print(i)

,count,mean,std,min,25%,50%,75%,max
question,5621.0,2.000000,0.000000,2.0,2.0,2.0,2.0,2.0
choice,5621.0,1.769080,0.724786,1.0,1.0,2.0,2.0,3.0
user,5621.0,837.630671,442.017450,1.0,437.0,880.0,1229.0,1531.0
age,5621.0,30.380715,9.284254,13.0,23.0,29.0,37.0,63.0
gender,5621.0,0.396904,0.489299,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...
OD_traffic_light_izq,5621.0,0.023483,0.277540,0.0,0.0,0.0,0.0,14.0
OD_train,5621.0,0.000178,0.306222,-4.0,0.0,0.0,0.0,4.0
OD_truck_der,5621.0,0.208682,0.603403,0.0,0.0,0.0,0.0,8.0
OD_truck_izq,5621.0,0.205124,0.623743,0.0,0.0,0.0,0.0,9.0


In [5]:
# car_dummies = pd.get_dummies(pd.cut(data['OD_car'],bins=[-20,-3,-1,0,2,20],labels=['OD_Car_20','OD_Car_10','OD_Car_00','OD_Car_01','OD_Car_02']))
# car_dummies = pd.get_dummies(pd.cut(data['OD_car'],bins=[-20,-1,0,20],labels=['OD_Car_10','OD_Car_00','OD_Car_01']))
car_dummies_izq = pd.get_dummies(pd.cut(data['OD_car_izq'],bins=[0,1,2,3,4,5,6,7,8,9,10,15,20,47],
                                    labels=['OD_Car_01_izq','OD_Car_02_izq','OD_Car_03_izq','OD_Car_04_izq','OD_Car_05_izq','OD_Car_06_izq','OD_Car_07_izq','OD_Car_08_izq','OD_Car_09_izq','OD_Car_10_izq',
                                            'OD_Car_15_izq','OD_Car_20_izq','OD_Car_40_izq']))
car_dummies_der = pd.get_dummies(pd.cut(data['OD_car_der'],bins=[0,1,2,3,4,5,6,7,8,9,10,15,20,47],
                                    labels=['OD_Car_01_der','OD_Car_02_der','OD_Car_03_der','OD_Car_04_der','OD_Car_05_der','OD_Car_06_der','OD_Car_07_der','OD_Car_08_der','OD_Car_09_der','OD_Car_10_der',
                                            'OD_Car_15_der','OD_Car_20_der','OD_Car_40_der']))
# # aux = pd.DataFrame(data['OD_cars_izq'])
# # aux = aux.join(car_dummies)
# # aux[(aux['OD_cars_izq']==1)]
data = data.join(car_dummies_izq)
data = data.join(car_dummies_der)


In [6]:
database = db.Database("database",data)
from headers import *

In [25]:
# OD_veh_izq = DefineVariable('OD_veh_izq', 
#                             OD_cars_izq + OD_bus_izq + OD_motorcycle_izq + OD_truck_izq, database)
# OD_veh_der = DefineVariable('OD_veh_der', 
#                             OD_cars_der + OD_bus_der + OD_motorcycle_der + OD_truck_der, database)

B_mean_h = Beta('LLF: mean_h', 0, -100, 100, 0)
B_mean_l = Beta('LLF: mean_l', 0, -100, 100, 0)
B_mean_s = Beta('LLF: mean_s', 0, -100, 100, 0)
B_mean_bn = Beta('LLF: mean_bn', 0, -100, 100, 1)  # NS
B_stdev_h = Beta('LLF: stdev_h', 0, -100, 100, 0)
B_stdev_l = Beta('LLF: stdev_l', 0, -100, 100, 0)
B_stdev_s = Beta('LLF: stdev_s', 0, -100, 100, 0)
B_stdev_bn = Beta('LLF: stdev_bn', 0, -100, 100, 1)  # NS

B_edges = Beta('LLF: edges', 0, -100, 100, 0)
B_blobs = Beta('LLF: blobs', 0, -100, 100, 1)


B_Bicyclist_N = Beta('HLF N: Bicyclist', 0, -100, 100, 0) 
B_Building_N = Beta('HLF N: Building', 0, -100, 100, 1)
B_Car_N = Beta('HLF N: Car', 0, -100, 100, 0) 
B_Fence_N = Beta('HLF N: Fence', 0, -100, 100, 0) 
B_Pavement_N = Beta('HLF N: Pavement', 0, -100, 100, 1)  # NS 0
B_Pedestrian_N = Beta('HLF N: Pedestrian', 0, -100, 100, 1)
B_Pole_N = Beta('HLF N: Pole', 0, -100, 100, 0) 
B_Road_N = Beta('HLF N: Road', 0, -100, 100, 0)  # NS 0
B_SignSymbol_N = Beta('HLF N: SignSymbol', 0, -100, 100, 0) 
B_Sky_N = Beta('HLF N: Sky', 0, -100, 100, 1)
B_Tree_N = Beta('HLF N: Tree', 0, -100, 100, 0)

B_Bicyclist_S = Beta('HLF S: Bicyclist', 0, -100, 100, 1) 
B_Building_S = Beta('HLF S: Building', 0, -100, 100, 1) 
B_Car_S = Beta('HLF S: Car', 0, -100, 100, 1)
B_Fence_S = Beta('HLF S: Fence', 0, -100, 100, 0) 
B_Pavement_S = Beta('HLF S: Pavement', 0, -100, 100, 1)
B_Pedestrian_S = Beta('HLF S: Pedestrian', 0, -100, 100, 1)
B_Pole_S = Beta('HLF S: Pole', 0, -100, 100, 0)
B_Road_S = Beta('HLF S: Road', 0, -100, 100, 0)
B_SignSymbol_S = Beta('HLF S: SignSymbol', 0, -100, 100, 1)
B_Sky_S = Beta('HLF S: Sky', 0, -100, 100, 1) # NS 0 
B_Tree_S = Beta('HLF S: Tree', 0, -100, 100, 0) 


D_bench = Beta('OD: bench', 0, -100, 100, 1)
D_boat = Beta('OD: boat', 0, -100, 100, 1)
D_umbrella = Beta('OD: umbrella', 0, -100, 100, 1)
D_chair = Beta('OD: chair', 0, -100, 100, 1) ## SI VA

D_train = Beta('OD: train', 0, -100, 100, 1)## SI VA
D_potted_plant = Beta('OD: potted_plant', 0, -100, 100, 1)
D_stop_sign = Beta('OD: stop_sign', 0, -100, 100, 1)
D_traffic_light = Beta('OD: traffic_light', 0, -100, 100, 1)

D_bus = Beta('OD: bus', 0, -100, 100, 1)
D_motorcycle = Beta('OD: motorcycle', 0, -100, 100, 1)
D_bike = Beta('OD: bike BC', 0, -100, 100, 0)
LAMBDA_bike = Beta('LAMBDA: bike',1.5,0.0001,5,0)## SI VA

# D_fire_hydrant = Beta('OD: fire_hydrant', 0, -100, 100, 1)

# D_truck = Beta('OD: truck', 0, -100, 100, 0)
# LAMBDA_truck = Beta('LAMBDA: truck',1,-5,5, 0)## SI VA

# D_bike = Beta('OD: bike', 0, -100, 100, 0)
# LAMBDA_bike = Beta('LAMBDA: bike',1,-5,5, 1)## SI VA

# D_car = Beta('OD: car', 0, -100, 100, 0)## SI VA
# LAMBDA_car = Beta('LAMBDA: car',1,0.1,15, 0)## SI VA

# D_person = Beta('OD: person', -0.0691, -100, 100, 1)## SI VA
# LAMBDA_person = Beta('LAMBDA: person',1,0.0001,5,1)## SI VA

# D_light = Beta('OD: light', 0, -100, 100, 1)
# LAMBDA_light = Beta('LAMBDA: light',1.0,0.0001,5,1)

D_fire_hydrant = Beta('OD: fire_hydrant', 0, -100, 100, 1)

D_truck = Beta('OD: truck', 0, -100, 100, 0)
LAMBDA_truck = Beta('LAMBDA: truck',1,-5,5, 0)## SI VA

D_bike = Beta('OD: bike', 0, -100, 100, 0)
LAMBDA_bike = Beta('LAMBDA: bike',1,-5,5, 1)## SI VA

D_car = Beta('OD: car', 0, -100, 100, 0)## SI VA
LAMBDA_car = Beta('LAMBDA: car',1,0.1,15, 0)## SI VA

D_person = Beta('OD: person', -0.0691, -100, 100, 1)## SI VA
LAMBDA_person = Beta('LAMBDA: person',1,0.0001,5,1)## SI VA

D_light = Beta('OD: light', 0, -100, 100, 1)
LAMBDA_light = Beta('LAMBDA: light',1.0,0.0001,5,1)

D_car_01 = Beta('OD: car 1', 0, -100, 100, 0)
D_car_02 = Beta('OD: car 2', 0, -100, 100, 0)
D_car_03 = Beta('OD: car 3', 0, -100, 100, 0)
D_car_04 = Beta('OD: car 4', 0, -100, 100, 0)
D_car_05 = Beta('OD: car 5', 0, -100, 100, 0)
D_car_06 = Beta('OD: car 6', 0, -100, 100, 0)
D_car_07 = Beta('OD: car 7', 0, -100, 100, 0)
D_car_08 = Beta('OD: car 8', 0, -100, 100, 0)
D_car_09 = Beta('OD: car 9', 0, -100, 100, 0)
D_car_10 = Beta('OD: car 10', 0, -100, 100, 0)
D_car_15 = Beta('OD: car 15', 0, -100, 100, 0)
D_car_20 = Beta('OD: car 20', 0, -100, 100, 0)
D_car_40 = Beta('OD: car 40+', 0, -100, 100, 0)


tau1	 = Beta('tau1',-1,None,0,0)
delta2	 = Beta('delta2',2,0,None,0)

tau2 = tau1 + delta2

U = B_mean_h * mean_h + \
     B_mean_l * mean_l + \
     B_mean_s * mean_s + \
     B_mean_bn * mean_bn + \
     B_stdev_h * stdev_h + \
     B_stdev_l * stdev_l + \
     B_stdev_s * stdev_s + \
     B_stdev_bn * stdev_bn + \
     B_edges * edges + \
     B_blobs * blobs + \
     B_Bicyclist_N * N_Bicyclist + \
     B_Building_N * N_Building + \
     B_Car_N * N_Car + \
     B_Fence_N * N_Fence + \
     B_Pavement_N * N_Pavement + \
     B_Pedestrian_N * N_Pedestrian + \
     B_Pole_N * N_Pole + \
     B_Road_N * N_Road + \
     B_SignSymbol_N * N_SignSymbol + \
     B_Sky_N * N_Sky + \
     B_Tree_N * N_Tree + \
     B_Bicyclist_S * S_Bicyclist + \
     B_Building_S * S_Building + \
     B_Car_S * S_Car + \
     B_Fence_S * S_Fence + \
     B_Pavement_S * S_Pavement + \
     B_Pedestrian_S * S_Pedestrian + \
     B_Pole_S * S_Pole + \
     B_Road_S * S_Road + \
     B_SignSymbol_S * S_SignSymbol + \
     B_Sky_S * S_Sky + \
     B_Tree_S * S_Tree + \
     D_bench * OD_bench + \
     D_chair * OD_chair + \
     D_fire_hydrant * OD_fire_hydrant + \
     D_potted_plant * OD_potted_plant + \
     D_stop_sign * OD_stop_sign + \
     D_train * OD_train + \
     D_truck * (OD_truck_der) ** LAMBDA_truck - \
     D_truck * (OD_truck_izq) ** LAMBDA_truck + \
     D_person * (OD_person_der)**LAMBDA_person - \
     D_person * (OD_person_izq)**LAMBDA_person + \
     D_car * (OD_car_der)**LAMBDA_car - \
     D_car * (OD_car_izq)**LAMBDA_car + \
     D_light * (OD_traffic_light_der) ** LAMBDA_light - \
     D_light * (OD_traffic_light_izq) ** LAMBDA_light + \
     D_bike * (OD_bicycle_der) ** LAMBDA_bike - \
     D_bike * (OD_bicycle_izq) ** LAMBDA_bike 

# + \
#      D_car_01 * ( OD_Car_01_der - OD_Car_01_izq ) + \
#      D_car_02 * ( OD_Car_02_der - OD_Car_02_izq ) + \
#      D_car_03 * ( OD_Car_03_der - OD_Car_03_izq ) + \
#      D_car_04 * ( OD_Car_04_der - OD_Car_04_izq ) + \
#      D_car_05 * ( OD_Car_05_der - OD_Car_05_izq ) + \
#      D_car_06 * ( OD_Car_06_der - OD_Car_06_izq ) + \
#      D_car_07 * ( OD_Car_07_der - OD_Car_07_izq ) + \
#      D_car_08 * ( OD_Car_08_der - OD_Car_08_izq ) + \
#      D_car_09 * ( OD_Car_09_der - OD_Car_09_izq ) + \
#      D_car_10 * ( OD_Car_10_der - OD_Car_10_izq ) + \
#      D_car_15 * ( OD_Car_15_der - OD_Car_15_izq ) + \
#      D_car_20 * ( OD_Car_20_der - OD_Car_20_izq ) + \
#      D_car_40 * ( OD_Car_40_der - OD_Car_40_izq ) 

# + \
#      D_bike * ( OD_bicycle_izq - OD_bicycle_der)


#      D_car_boxcox * models.boxcox(OD_car_izq,LAMBDA_car) - \
#      D_car_boxcox * models.boxcox(OD_car_der,LAMBDA_car ) 


In [26]:
P1 = 1-dist.logisticcdf(U-tau1)
P2 = dist.logisticcdf(U-tau2)
Pe = dist.logisticcdf(U-tau1)- dist.logisticcdf(U-tau2)

ChoiceProba = {
    1: P1,
    3: Pe,
    2: P2}

# bioNormalCdf
# logisticcdf

logprob = log(Elem(ChoiceProba,choice))
formulas = {'loglike':logprob,
           'Prob 1':P1,
           'Prob =':Pe,
           'Prob 2':P2,
           'Utilidad':U}

biogeme = bio.BIOGEME(database,formulas,numberOfThreads=8)

biogeme.modelName = "live - OD"
# biogeme.cfsqp(0,0)
# results = biogeme.estimate()

## algorithms
results = biogeme.estimate(algorithm=None)
R = results.getEstimatedParameters()

# B = pd.DataFrame(R['Value'])
# T = pd.DataFrame(R['t-test'])

# B = pd.concat([B,R['Value']],axis=1, sort=False)
# T = pd.concat([T,R['t-test']],axis=1, sort=False)

In [27]:
B = pd.DataFrame(R['Value'])
# T = pd.DataFrame(R['t-test'])
T = pd.DataFrame(R['Rob. t-test'])

In [28]:
indice = pd.Index(['HLF N: Bicyclist',
                 'HLF N: Building',
                 'HLF N: Car',
                 'HLF N: Fence',
                 'HLF N: Pavement',
                 'HLF N: Pedestrian',
                 'HLF N: Pole',
                 'HLF N: Road',
                 'HLF N: SignSymbol',
                 'HLF N: Sky',
                 'HLF N: Tree',
                 'HLF S: Bicyclist',
                 'HLF S: Building',
                 'HLF S: Car',
                 'HLF S: Fence',
                 'HLF S: Pavement',
                 'HLF S: Pedestrian',
                 'HLF S: Pole',
                 'HLF S: Road',
                 'HLF S: SignSymbol',
                 'HLF S: Sky',
                 'HLF S: Tree',
                 'LLF: edges',
                 'LLF: mean_h',
                 'LLF: mean_l',
                 'LLF: mean_s',
                 'LLF: stdev_h',
                 'LLF: stdev_l',
                 'LLF: stdev_s',
                 'delta2',
                 'tau1',
                 'OD: bike',
                 'OD: car',
                 'OD: light',
                 'OD: person',
                 'OD: train',
                 'OD: truck',
                 'LAMBDA: car',
                 'LAMBDA: person',
                 'LAMBDA: person'])
show = pd.DataFrame(index=indice)

pd.concat([show,B.iloc[:,0],T.iloc[:,0]], axis=1).fillna('')

C:\Users\Tom\Anaconda3\envs\bio\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Value,Rob. t-test
HLF N: Bicyclist,6.18925,2.18711
HLF N: Building,,
HLF N: Car,-0.676633,-1.79584
HLF N: Fence,0.83676,2.70882
HLF N: Pavement,,
HLF N: Pedestrian,,
HLF N: Pole,-1.61182,-2.90277
HLF N: Road,-1.90557,-3.42635
HLF N: SignSymbol,1.50457,3.50539
HLF N: Sky,,


In [ ]:
# model = 'live_Wekun~OD Cars exp.pickle'
# # model = 'live_Wekun~base SEGNET~00.pickle'
# results = res.bioResults(pickleFile = model)

# betas = dict()
# for i in range(len(results.data.betaValues)):
#     betas[results.data.betaNames[i]] = results.data.betaValues[i]

# simulation = biogeme.simulate(betas)
# R = results.getEstimatedParameters()

In [105]:
T

,t-test,t-test,t-test,t-test
HLF N: Fence,3.652723,3.522543,3.566975,2.596216
HLF N: Pavement,-4.280898,-4.232234,-4.541583,NaN
HLF N: Pole,-2.534479,-2.419038,-2.508416,-2.360935
HLF N: SignSymbol,1.840382,1.780644,1.886202,2.141265
HLF S: Car,1.975662,1.977219,1.933247,NaN
HLF S: Fence,5.274429,5.277248,5.038657,5.220971
HLF S: Pavement,2.149768,2.134666,2.161178,NaN
HLF S: Pole,-2.921378,-2.806716,-2.870905,-2.963350
HLF S: Road,10.313475,10.368847,10.300517,9.885644
HLF S: Tree,8.962738,8.889686,8.506817,8.368022


In [106]:
B

,Value,Value,Value,Value
HLF N: Fence,1.201833,1.156288,1.174564,0.840416
HLF N: Pavement,-2.426754,-2.395617,-2.574395,NaN
HLF N: Pole,-1.432672,-1.364956,-1.420017,-1.381897
HLF N: SignSymbol,0.801338,0.774889,0.822602,0.938914
HLF S: Car,0.586161,0.586434,0.574557,NaN
HLF S: Fence,1.687380,1.687667,1.612309,1.671930
HLF S: Pavement,0.995973,0.988629,1.002646,NaN
HLF S: Pole,-1.639655,-1.572454,-1.613204,-1.657847
HLF S: Road,5.178091,5.204894,5.175676,4.746161
HLF S: Tree,2.438757,2.416800,2.301060,2.241174


In [39]:
indice = pd.Index(['HLF N: Bicyclist',
                 'HLF N: Building',
                 'HLF N: Car',
                 'HLF N: Fence',
                 'HLF N: Pavement',
                 'HLF N: Pedestrian',
                 'HLF N: Pole',
                 'HLF N: Road',
                 'HLF N: SignSymbol',
                 'HLF N: Sky',
                 'HLF N: Tree',
                 'HLF S: Bicyclist',
                 'HLF S: Building',
                 'HLF S: Car',
                 'HLF S: Fence',
                 'HLF S: Pavement',
                 'HLF S: Pedestrian',
                 'HLF S: Pole',
                 'HLF S: Road',
                 'HLF S: SignSymbol',
                 'HLF S: Sky',
                 'HLF S: Tree',
                 'LAMBDA: car',
                 'LAMBDA: person',
                 'LLF: edges',
                 'LLF: mean_h',
                 'LLF: mean_l',
                 'LLF: mean_s',
                 'LLF: stdev_h',
                 'LLF: stdev_l',
                 'LLF: stdev_s',
                 'OD: car BC',
                 'OD: person BC',
                 'delta2',
                 'tau1'])
show = pd.DataFrame(index=indice)
show.join([B.iloc[-1],T.iloc[-1]])

,tau1_x,tau1_y
HLF N: Bicyclist,NaN,NaN
HLF N: Building,NaN,NaN
HLF N: Car,NaN,NaN
HLF N: Fence,NaN,NaN
HLF N: Pavement,NaN,NaN
HLF N: Pedestrian,NaN,NaN
HLF N: Pole,NaN,NaN
HLF N: Road,NaN,NaN
HLF N: SignSymbol,NaN,NaN
HLF N: Sky,NaN,NaN


In [ ]:
indice = pd.Index(['HLF N: Bicyclist',
                 'HLF N: Building',
                 'HLF N: Car',
                 'HLF N: Fence',
                 'HLF N: Pavement',
                 'HLF N: Pedestrian',
                 'HLF N: Pole',
                 'HLF N: Road',
                 'HLF N: SignSymbol',
                 'HLF N: Sky',
                 'HLF N: Tree',
                 'HLF S: Bicyclist',
                 'HLF S: Building',
                 'HLF S: Car',
                 'HLF S: Fence',
                 'HLF S: Pavement',
                 'HLF S: Pedestrian',
                 'HLF S: Pole',
                 'HLF S: Road',
                 'HLF S: SignSymbol',
                 'HLF S: Sky',
                 'HLF S: Tree',
                 'LAMBDA: car',
                 'LAMBDA: person',
                 'LLF: edges',
                 'LLF: mean_h',
                 'LLF: mean_l',
                 'LLF: mean_s',
                 'LLF: stdev_h',
                 'LLF: stdev_l',
                 'LLF: stdev_s',
                 'OD: car BC',
                 'OD: person BC',
                 'delta2',
                 'tau1'])
show = pd.DataFrame(index=indice)
show.join([B,T])

In [12]:
pd.concat([B.iloc[:,0],T.iloc[:,0]],axis =1)

,Value,t-test
HLF N: Fence,-0.859572,-2.658355e+00
HLF N: Pole,1.238750,2.111814e+00
HLF N: Road,2.010022,3.330062e+00
HLF N: SignSymbol,-0.891129,-2.034150e+00
HLF N: Tree,-0.502771,-1.181574e+00
HLF S: Car,-0.296942,-1.012893e+00
HLF S: Fence,-1.806029,-5.604506e+00
HLF S: Pavement,-0.817212,-1.722714e+00
HLF S: Pole,1.500146,2.688913e+00
HLF S: Road,-5.122413,-1.011567e+01


In [25]:
simulation.head()

,loglike,Prob 1,Prob =,Prob 2,Utilidad
2,-1.198194,0.301739,0.190997,0.507264,0.389645
11,-1.660322,0.512986,0.190078,0.296937,-0.501342
12,-0.387787,0.174060,0.147383,0.678557,1.107733
16,-1.217611,0.295936,0.189880,0.514184,0.417336
17,-0.940153,0.409738,0.199694,0.390568,-0.084339


In [26]:
pr = simulation.drop(columns=['Utilidad'])
maxpr = pd.DataFrame(pr.idxmax(axis=1))
maxpr = maxpr.join(data['choice'])
maxpr[0] = maxpr[0].replace({'Prob 2':2,'Prob 1':1,'Prob =':3})

In [27]:
## Excluir indiferencias
FPR_df = maxpr[(maxpr['choice']!=3)]
FPR = FPR_df[(FPR_df[0]== FPR_df['choice'])].shape[0]/FPR_df.shape[0]
FPR

0.7028093800789412

In [62]:
maxpr[(maxpr[0]==maxpr['choice'])]

,0,choice
14,2,2
25,2,2
26,2,2
54,1,1
57,2,2
...,...,...
25153,1,1
25158,1,1
25167,2,2
25170,2,2


In [89]:
4163/7264*100

57.31002202643172

In [1]:
help(bio)

NameError: name 'bio' is not defined

In [ ]:
-0.1052-(-0.1909)

In [ ]:
-0.1909-(-0.2936)

In [ ]:
-0.2936-(-0.3173)

In [ ]:
# T.to_csv('test_t.csv')
# B.to_csv('betas.csv')

In [34]:
import biogeme.results as res
results = res.bioResults(pickleFile ='walk_Wekun~OD Final.pickle')

AttributeError: 'bioResults' object has no attribute 'Variables'

In [ ]:
# help(results)

In [ ]:
# var = data['Tree_N']
# var.plot.hist(bins=50)